# Tokenization comparison in ViViT
## Uniform Frame Sampling vs. Tubelet Embedding

Since 3D vision datasets (videos, 3D images, etc) have additional axis, tokenizing method can be another challenge for efficient model. Thus here is a comparison between **Uniform Frame Sampling** and **Tubelet Embedding** both introduced in paper [ViViT: A Video Vision Transformer](https://arxiv.org/abs/2103.15691).

### Settings 
This experiment is done on 6 datasets from **medmnist3D**, where all of them has 28 time periods, 28 pixels in height, 28 pixels in width, and 1 channel. Hyperparameters were fixed except for learning rate.

### The Result
Overall performance of Tubelet Embedding excelled Uniform Frame Sampling. Also, the time elapsed for each training was shorter for Tubelete Embedding method. The **full result of this note book** is on **[TensorBoard.dev](https://tensorboard.dev/experiment/PKs2SEeNQLO68B4tB7U8tg/)**.

### Limitations
Since the experiment does not cover variations of dataset distribution(on image size, length of time period, types of data) and hyperparameters on ViViT architecture, _this is not a general consequence of the tokenization methods._ However, given the limitations of operation the simple setting and the result can be an intuitive base for those who are curious :-)

### Reference
1. Datasets : https://medmnist.com/
1. Paper : https://arxiv.org/abs/2103.15691
1. Codes : https://keras.io/examples/vision/vivit/


## Install & Load packages

In [1]:
!pip install -q tensorflow-addons
!pip install -q tensorflow-gpu
!pip install -q -U tensorboard
!pip install -q medmnist

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tensorflow-gpu
  error: subprocess-exited-with-error
  
  × Running setup.py install for tensorflow-gpu did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Running setup.py install for tensorflow-gpu ... error
error: legacy-install-failure

× Encountered error while trying to install package.
╰─> tensorflow-gpu

note: This is an issue with the package mentioned above, not pip.
hint: See above for output from the failure.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 38.4 MB/s eta 0:00

In [2]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [3]:
import os
import io
import time
import imageio
import medmnist
import ipywidgets
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow_addons as tfa
from tensorflow.keras import layers

import matplotlib.pyplot as plt
from datetime import datetime

# Setting seed for reproducibility
SEED = 2023
os.environ["TF_CUDNN_DETERMINISTIC"] = "1"
keras.utils.set_random_seed(SEED)

In [4]:
# Check device
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

if tf.config.list_physical_devices('GPU'):
    DEVICE_NAME = "/gpu:0"
    print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
else:  
    DEVICE_NAME = "/cpu:0"

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3709101813517062318
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 13912375296
locality {
  bus_id: 1
  links {
  }
}
incarnation: 2596457658420606362
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
xla_global_id: 416903419
]
Num GPUs Available:  1


In [5]:
# Refrain from verbose logging
import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

### Hyperparameters

In [6]:
# DATA
# source : https://github.com/MedMNIST/MedMNIST/blob/main/medmnist/dataset.py
DATASET_INFOS = (("organmnist3d", 11), 
                 ("nodulemnist3d", 2),
                 ("adrenalmnist3d", 2),
                 ("fracturemnist3d", 3),
                 ("vesselmnist3d", 2),
                 ("synapsemnist3d", 2))  # (dataset_name, num_classes)
BATCH_SIZE = 64
AUTO = tf.data.AUTOTUNE
INPUT_SHAPE = (28, 28, 28, 1)

# OPTIMIZER
LEARNING_RATES = [1e-3, 1e-4, 1e-5]
WEIGHT_DECAY = 1e-5

# TRAINING
EPOCHS = 60

# EMBEDDING
PATCH_SIZE = 8
NUM_PATCHES = (INPUT_SHAPE[0] // PATCH_SIZE) ** 2 

# ViViT ARCHITECTURE
LAYER_NORM_EPS = 1e-6
PROJECTION_DIM = 128
NUM_HEADS = 8
NUM_LAYERS = 8
MLP_UNITS = [
    PROJECTION_DIM * 2,
    PROJECTION_DIM,
]

## Data Loading & Preparing

In [7]:
def download_and_prepare_dataset(data_info: dict):
    """Utility function to download the dataset.

    Arguments:
        data_info (dict): Dataset metadata.
    """
    data_path = keras.utils.get_file(
        origin=data_info["url"], md5_hash=data_info["MD5"]
    )

    with np.load(data_path) as data:
        # Get videos
        train_videos = data["train_images"]
        valid_videos = data["val_images"]
        test_videos = data["test_images"]

        # Get labels
        train_labels = data["train_labels"].flatten()
        valid_labels = data["val_labels"].flatten()
        test_labels = data["test_labels"].flatten()

    return (
        (train_videos, train_labels),
        (valid_videos, valid_labels),
        (test_videos, test_labels),
    )

In [8]:
@tf.function
def preprocess(frames: tf.Tensor, label: tf.Tensor):
    """Preprocess the frames tensors and parse the labels."""
    # Preprocess images
    frames = tf.image.convert_image_dtype(
        frames[
            ..., tf.newaxis
        ],  # The new axis is to help for further processing with Conv3D layers
        tf.float32,
    )
    # Parse label
    label = tf.cast(label, tf.float32)
    return frames, label


def prepare_dataloader(
        videos: np.ndarray,
        labels: np.ndarray,
        loader_type: str = "train",
        batch_size: int = BATCH_SIZE,
    ):
    """Utility function to prepare the dataloader."""
    dataset = tf.data.Dataset.from_tensor_slices((videos, labels))

    if loader_type == "train":
        dataset = dataset.shuffle(BATCH_SIZE * 2)

    dataloader = (
        dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )
    return dataloader


def get_dataloaders(dataset_name):
    # Get the metadata of the dataset
    info = medmnist.INFO[dataset_name]

    # Get the dataset
    prepared_dataset = download_and_prepare_dataset(info)
    (train_videos, train_labels) = prepared_dataset[0]
    (valid_videos, valid_labels) = prepared_dataset[1]
    (test_videos, test_labels) = prepared_dataset[2]

    # Prepare DataLoaders
    trainloader = prepare_dataloader(train_videos, train_labels, "train")
    validloader = prepare_dataloader(valid_videos, valid_labels, "valid")
    testloader = prepare_dataloader(test_videos, test_labels, "test")

    return (trainloader, validloader, testloader)

## Define Structures
### Define Embedding Methods

In [9]:
# Define Uniform Frame Sampling, which is an extension of 2D Embedding method.
class UniformFrameSampling(layers.Layer):
    def __init__(self, embed_dim=PROJECTION_DIM, patch_size=PATCH_SIZE, **kwargs):
        super().__init__(**kwargs)
        self.patch_size = patch_size
        self.projection2d = layers.Conv2D(
            filters=embed_dim,
            kernel_size=(patch_size, patch_size),
            strides=(patch_size, patch_size),
            padding="valid"
        )
        self.concat = layers.Concatenate(axis=1)
        self.flatten = layers.Reshape(target_shape=(-1, embed_dim))

    def call(self, videos):
        t = videos.shape[1]
        projected_patches = []
        for frame in range(self.patch_size, t, self.patch_size): # Sample n_t
            patch = self.projection2d(videos[:, frame])  # (B, n_h, n_w, embed_dim)
            _, n_h, n_w, embed_dim = patch.shape
            patch = layers.Reshape((-1, 1, n_h, n_w, embed_dim))(patch)
            projected_patches.append(patch)
        projected_patches = self.concat(projected_patches)  # (B, n_t, n_h, n_w, embed_dim)
        flattened_patches = self.flatten(projected_patches)  # (B, num_patches, embed_dim)
        return flattened_patches

In [10]:
# Define Tubelet Embedding, which is a time-spatial embedding for video.
class TubeletEmbedding(layers.Layer):
    def __init__(self, embed_dim=PROJECTION_DIM, patch_size=PATCH_SIZE, **kwargs):
        super().__init__(**kwargs)
        self.projection3d = layers.Conv3D(
            filters=embed_dim,
            kernel_size=(patch_size, patch_size, patch_size),
            strides=(patch_size, patch_size, patch_size),
            padding="valid"
        )
        self.flatten = layers.Reshape(target_shape=(-1, embed_dim))

    def call(self, videos):  # videos.shape = (B, T, H, W, C)
        projected_patches = self.projection3d(videos)  # (B, n_t, n_h, n_w, embed_dim)
        flattened_patches = self.flatten(projected_patches)  # (B, num_patches, embed_dim)
        return flattened_patches

In [11]:
# Positionnal Encoding for both embedding methods.
class PositionalEncoder(layers.Layer):
    def __init__(self, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim

    def build(self, input_shape):
        _, num_tokens, _ = input_shape
        self.position_embedding = layers.Embedding(
            input_dim=num_tokens, output_dim=self.embed_dim
        )
        self.positions = tf.range(start=0, limit=num_tokens, delta=1)

    def call(self, encoded_tokens):
        # Encode the positions and add it to the encoded tokens
        encoded_positions = self.position_embedding(self.positions)
        encoded_tokens = encoded_tokens + encoded_positions
        return encoded_tokens

### Define Model

In [12]:
def create_vivit_classifier(
    embedder,
    positional_encoder,
    num_classes,
    input_shape=INPUT_SHAPE,
    transformer_layers=NUM_LAYERS,
    num_heads=NUM_HEADS,
    embed_dim=PROJECTION_DIM,
    layer_norm_eps=LAYER_NORM_EPS,
):
    # Get the input layer
    inputs = layers.Input(shape=input_shape)
    # Create patches.
    patches = embedder(inputs)
    # Encode patches.
    encoded_patches = positional_encoder(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization and MHSA
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim // num_heads, dropout=0.1
        )(x1, x1)

        # Skip connection
        x2 = layers.Add()([attention_output, encoded_patches])

        # Layer Normalization and MLP
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        x3 = keras.Sequential(
            [
                layers.Dense(units=embed_dim * 4, activation=tf.nn.gelu),
                layers.Dense(units=embed_dim, activation=tf.nn.gelu),
            ]
        )(x3)

        # Skip connection
        encoded_patches = layers.Add()([x3, x2])

    # Layer normalization and Global average pooling.
    representation = layers.LayerNormalization(epsilon=layer_norm_eps)(encoded_patches)
    representation = layers.GlobalAvgPool1D()(representation)

    # Classify outputs.
    outputs = layers.Dense(units=num_classes, activation="softmax")(representation)

    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

## Train
Log on TensorBoard

In [15]:
def run_experiment(dataset_info, learning_rate, use_tubelet_embedding=True):
    with tf.device(DEVICE_NAME):
        # Define dataset
        dataset_name, num_classes = dataset_info
        trainloader, validloader, testloader = get_dataloaders(dataset_name)

        # Define embedder
        if use_tubelet_embedding:
            embedder = TubeletEmbedding(
                embed_dim=PROJECTION_DIM, patch_size=PATCH_SIZE
            )
        else:
            embedder = UniformFrameSampling(
                embed_dim=PROJECTION_DIM, patch_size=PATCH_SIZE
            )

        model = create_vivit_classifier(
            embedder,
            positional_encoder=PositionalEncoder(embed_dim=PROJECTION_DIM),
            num_classes=num_classes
        )
        optimizer = tfa.optimizers.AdamW(
            learning_rate=learning_rate, weight_decay=WEIGHT_DECAY
        )
        model.compile(
            optimizer=optimizer,
            loss="sparse_categorical_crossentropy",
            metrics=[
                keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
                keras.metrics.SparseTopKCategoricalAccuracy(5, name="top-5-accuracy"),
            ],
        )
        start = time.time()

        # Define checkpoint callbacks
        checkpoint_filepath = "/tmp/checkpoint"
        checkpoint_callback = keras.callbacks.ModelCheckpoint(
            checkpoint_filepath,
            monitor="val_accuracy",
            save_best_only=True,
            save_weights_only=True,
        )

        # Define tensorboard callbacks
        embedder_name = "tubelet" if use_tubelet_embedding else "sampling"
        log_dir = f"logs/fit/{dataset_name}/{embedder_name}/lr:{learning_rate}/" \
                  + datetime.now().strftime("%Y%m%d-%H%M%S")
        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

        # Train the model
        _ = model.fit(
            trainloader,
            epochs=EPOCHS,
            validation_data=validloader,
            callbacks=[tensorboard_callback, checkpoint_callback],
        )

        model.load_weights(checkpoint_filepath)
        _, accuracy, top_5_accuracy = model.evaluate(testloader)
        print(f"*** Data: {dataset_name}, Embedder: {embedder_name}, LR: {learning_rate} ***")
        print(f"Test accuracy: {round(accuracy * 100, 2)}%")
        print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")
        print(f"Time elapsed: {round(time.time() - start, 4)}s")

In [16]:
# Clear out prior logging data.
!rm -rf logs/fit

for dataset_info in DATASET_INFOS:
    for learning_rate in LEARNING_RATES:
        run_experiment(dataset_info, learning_rate, use_tubelet_embedding=False)
        run_experiment(dataset_info, learning_rate, use_tubelet_embedding=True)

32657407/32657407 [==============================] - 6s 0us/step
Epoch 1/60
16/16 [==============================] - 20s 242ms/step - loss: 2.8413 - accuracy: 0.1060 - top-5-accuracy: 0.4949 - val_loss: 2.6553 - val_accuracy: 0.0994 - val_top-5-accuracy: 0.4783
Epoch 2/60
16/16 [==============================] - 5s 313ms/step - loss: 2.3832 - accuracy: 0.1029 - top-5-accuracy: 0.5319 - val_loss: 2.4320 - val_accuracy: 0.0994 - val_top-5-accuracy: 0.4969
Epoch 3/60
16/16 [==============================] - 3s 180ms/step - loss: 2.3265 - accuracy: 0.1132 - top-5-accuracy: 0.5566 - val_loss: 2.3706 - val_accuracy: 0.0932 - val_top-5-accuracy: 0.5031
Epoch 4/60
16/16 [==============================] - 2s 147ms/step - loss: 2.2502 - accuracy: 0.1101 - top-5-accuracy: 0.6224 - val_loss: 2.2205 - val_accuracy: 0.1988 - val_top-5-accuracy: 0.6584
Epoch 5/60
16/16 [==============================] - 2s 144ms/step - loss: 2.0875 - accuracy: 0.2233 - top-5-accuracy: 0.6944 - val_loss: 1.9099 - val_

## Check the result
Please refer to [here](https://tensorboard.dev/experiment/PKs2SEeNQLO68B4tB7U8tg/).

In [ ]:
# # Show tensorboard extension
# %tensorboard --logdir logs/fit

In [ ]:
# Upload the experiment on Tensorboard.dev
!tensorboard dev upload --logdir logs \
    --name "Tokenization comparison in ViViT" \
    --description "Simple comparison between Uniform Frame Sampling and Tubelet Embedding"


***** TensorBoard Uploader *****

This will upload your TensorBoard logs to https://tensorboard.dev/ from
the following directory:

logs

This TensorBoard will be visible to everyone. Do not upload sensitive
data.

Your use of this service is subject to Google's Terms of Service
<https://policies.google.com/terms> and Privacy Policy
<https://policies.google.com/privacy>, and TensorBoard.dev's Terms of Service
<https://tensorboard.dev/policy/terms/>.

This notice will not be shown again while you are logged into the uploader.
To log out, run `tensorboard dev auth revoke`.

Continue? (yes/NO) yes

To sign in with the TensorBoard uploader:

1. On your computer or phone, visit:

   https://www.google.com/device

2. Sign in with your Google account, then enter:

   SHZ-BRW-HVN


Upload started and will continue reading any new data as it's added to the logdir.

To stop uploading, press Ctrl-C.

New experiment created. View your TensorBoard at: https://tensorboard.dev/experiment/PKs2SEeNQLO